In [1]:
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from pyscf.geomopt.berny_solver import optimize
from berny import Berny, geomlib,Geometry,Math
import berny.coords
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM,FcM_like
from alch_deriv import alch_deriv
from Morse import mpa, Morse_interpolator
from AP_class import APDFT_perturbator as AP
from FDcoeffs import *
from ABSE import absec
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
def nr_step(ic,g,h):
    ncoords=ic.shape[0]*ic.shape[1]
    p1cds=ic-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1)).reshape(ic.shape[0],ic.shape[1])
    p1cds-=p1cds[0]
    print (norm(p1cds[1]))
    if p1cds.shape[0]>2:
        print (anglecenter(p1cds),anglecenter(p1cds)*np.pi/180)
    return(p1cds)

In [42]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic

ebde=100/627.5 # empirical bond dissociation energy 100 kcal

def mpa_pb(coords,atoml,g,h,e,gic=False,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
    h_ic=build_h_ic(s,g_ic,h,B_inv)
    bnr=0
    dde_mb=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                MorseInt=Morse_interpolator(g_ic[i],h_ic[i,i],q0[i],ebde,e)
                ddq_mb[i]+=MorseInt.re+g_ic[i]/h_ic[i,i]-q0[i]
                dde_mb+=MorseInt.e_min-e+1/2*h_ic[i,i]*(MorseInt.re-q0[i])**2

    dq_NR=-np.linalg.solve(h_ic,g_ic)
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:bnr]@(ddq_mb[:bnr]))
    dq_tot=dq_NR+ddq_mb
    en_MB=(e-1/2*np.einsum("i,ij,j",dq_tot,h_ic,dq_tot)+dde_mb)
    e_NR=(e-1/2*np.einsum("i,ij,j",dq_tot,h_ic,dq_tot))
    print (s.coords._coords)
    return q0,q0+dq_tot,en_MB,e_NR

In [24]:
ch4coords,nh3optg,oh2optg,hfoptg=np.load("../optgeoms.npy")
nh3=gto.M(atom="N "+str(nh3optg[0])[1:-1]+";H "+str(nh3optg[1])[1:-1]+";H "+str(nh3optg[2])[1:-1]+
          ";H "+str(nh3optg[3])[1:-1],basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},unit="Bohr")


In [13]:
scf.RHF(nh3).scf()

converged SCF energy = -56.2221836146944


-56.22218361469435

In [14]:
nh3_2=gto.M(atom="N 0. 0. 0.;H 3.90982817e-13 3.71707821e-01 1.84851805e+00; H  1.52736243  0.80139969 -0.76164836;H -1.52736243  0.80139969 -0.76164836",
          basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},unit="Bohr")

mf=scf.RHF(nh3_2)

In [15]:
mf.scf()

converged SCF energy = -56.2221836146943


-56.22218361469432

In [34]:
nh3_2=gto.M(atom="N 0. 0. 0.;H 0 3.91707821e-01 2.0e+00; H  2  0.70139969 -0.76164836;H -1.7  0.40139969 -0.56164836",
          basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},unit="Bohr")

mf=scf.RHF(nh3_2)

In [35]:
e_0=mf.scf()
g_0=mf.Gradients().grad()
h_0=mf.Hessian().hess()

converged SCF energy = -56.1794279051978
--------------- RHF gradients ---------------
         x                y                z
0 N    -0.1171399144    -0.0114869805    -0.0684129468
1 H    -0.0047093709     0.0120154107     0.0592830557
2 H     0.1125993926     0.0216071530    -0.0229335607
3 H     0.0092498926    -0.0221355831     0.0320634517
----------------------------------------------


In [40]:
mpa_pb(nh3_2.atom_coords(),['N','H','H','H'],g_0,h_0,e_0)

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(array([2.0379978 , 2.25212561, 1.83482168, 1.84629428, 1.83211043,
        2.27988418]),
 array([1.88738495, 2.01318295, 1.87306585, 1.84673394, 1.87640548,
        1.59337029]),
 -56.225426356938286,
 -56.20834670072265)

In [43]:
mpa_pb(nh3_2.atom_coords(),['N','H','H','H'],g_0,h_0,e_0)

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(array([2.0379978 , 2.25212561, 1.83482168, 1.84629428, 1.83211043,
        2.27988418]),
 array([1.88738495, 2.01318295, 1.87306585, 1.84673394, 1.87640548,
        1.59337029]),
 -56.225426356938286,
 -56.2452574991359)

In [32]:
mpa_pb(nh3.atom_coords(),['N','H','H','H'],g_0,h_0,e_0)

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(array([1.88552   , 1.88552   , 1.88552   , 1.88833539, 1.88833539,
        1.88846925]),
 array([1.76039051, 1.86731482, 2.09824834, 1.76346751, 1.7776079 ,
        1.27459116]),
 -56.226137857055015,
 -56.22362283186789)